<img src="qiskit_logo.png">

# Basis Gate Decomposition of Arbitrary Single-Qubit Unitary Operators


## Contributors
Bruno Murta, Quantalab, International Iberian Nanotechnology Laboratory (INL)

## QISKit Package Versions

In [1]:
from qiskit import *
import numpy as np, cmath, sys, random
qiskit.__qiskit_version__

{'qiskit-terra': '0.9.0',
 'qiskit-ignis': '0.2.0',
 'qiskit-aqua': '0.6.0',
 'qiskit': '0.12.0',
 'qiskit-aer': '0.3.0',
 'qiskit-ibmq-provider': '0.3.2'}

## Introduction

Implementing quantum algorithms in quantum hardware requires their conversion into a quantum circuit. In practice, this means decomposing a unitary matrix into a sequence of basis gates. Accomplishing this for any unitary matrix is therefore a valuable skill in quantum computing.

In this notebook, it is explained how to find the basis gate decomposition of any 2x2 unitary matrix, as well as of the respective controlled version.

## Implementation of Arbitrary 2x2 Unitary Matrix via $U_3$ Gate

Most single-qubit gates used in quantum algorithms can be immediately written in terms of Clifford gates or rotations around the cartesian axes of the Bloch sphere. However, in general this decomposition may not be obvious. The QISKit basis gate $U_3$ allows to implement any 2x2 unitary matrix, though:

$$
\begin{split}
    & U_3(\theta, \phi, \lambda) = \left( \begin{matrix} \cos(\theta/2) & -e^{i \lambda} \sin(\theta/2) \\ e^{i\phi} \sin(\theta/2) & e^{i(\lambda + \phi)} \cos(\theta/2) \end{matrix} \right) 
\end{split}
$$

A general expression for a 2x2 unitary matrix is:

$$
\begin{split}
    & \mathcal{U} = \left( \begin{matrix} a & b \\ -e^{i\eta} b^{*} & e^{i\eta} a^{*} \end{matrix} \right),
\end{split}
$$

with $|a|^2 + |b|^2 = 1$ and $\textrm{det}(\mathcal{U}) = e^{i\eta}$. The two matrices can be equated (up to a global phase), giving:

$$\theta = 2 \arctan\Big(\frac{|b|}{|a|}\Big) \\
  \lambda = -\pi + \textrm{arg}(b) - \textrm{arg}(a)\\
  \phi = \pi + \eta - \textrm{arg}(a) - \textrm{arg}(b)$$

where $a = |a| \; e^{i\textrm{arg}(a)}$ and similarly for $b$. We can now define a function that generates this two-gate decomposition corresponding to a given 2x2 unitary matrix:

In [2]:
def is_unitary(A):
    '''
    Function that determines if A is a unitary matrix
    U: Unitary matrix of any size [numpy.ndarray]
    '''
    
    return np.allclose(np.eye(A.shape[0]), A.dot(np.transpose(np.conjugate(A))))

def find_params(U):
    '''
    Function that finds parameters of U3 that implement unitary matrix U up to global phase
    U: Unitary 2x2 matrix [numpy.ndarray]
    '''
    
    # Checking if input is numpy.ndarray
    if not type(U) is np.ndarray:
        sys.exit("Input must be numpy.ndarray")
    
    # Checking if input has right dimensionality
    if not U.shape == (2,2):
        sys.exit("Input must be 2x2 matrix")
        
    # Checking if input is unitary
    if not is_unitary(U):
        sys.exit("Input must be unitary")
    
    a = U[0,0]
    b = U[0,1]
    eta = np.angle(U[1,1]/(np.conjugate(a)))
    
    theta = 2*np.arctan(np.absolute(b)/np.absolute(a))
    lamda = -cmath.pi - np.angle(a) + np.angle(b)
    phi = cmath.pi + eta - np.angle(a) - np.angle(b)
    
    params = [theta, phi, lamda]
    
    return params

def u3_given_matrix(U):
    '''
    Function that outputs U3 gate that implement unitary matrix U
    U: Unitary 2x2 matrix [numpy.ndarray]
    '''
    
    [theta, phi, lamda] = find_params(U)
    
    qc = QuantumCircuit(1)
    qc.u3(theta,phi,lamda,0)
    
    # Sanity check to accuracy set by f_acc
    f_acc = 1e-6
    
    simulator = Aer.get_backend('unitary_simulator')
    result = execute(qc, simulator).result()
    u3_out = result.get_unitary(qc)
    
    numerator = (np.trace(np.matmul(np.conjugate(np.transpose(U)), u3_out)))**2
    denominator = np.trace(np.matmul(np.conjugate(np.transpose(U)), U)) * np.trace(np.matmul(np.conjugate(np.transpose(u3_out)), u3_out))
    fidelity_measure = numerator / denominator
        
    if 1 - abs(fidelity_measure) < f_acc:
        return qc
    else:
        sys.exit("Could not obtain ZYZ decomposition to desired accuracy")

Let us test this function for a randomly generated 2x2 unitary matrix:

In [8]:
U = qiskit.quantum_info.random_unitary(2).data
qc = u3_given_matrix(U)

# Sanity check
simulator = qiskit.Aer.get_backend('unitary_simulator')
result = execute(qc, simulator).result()
output_u3 = result.get_unitary(qc)

print(np.divide(output_u3, U))

[[-0.96918997+0.24631442j -0.96918997+0.24631442j]
 [-0.96918997+0.24631442j -0.96918997+0.24631442j]]


As expected, the original matrix and its corresponding $U_3$ gate only differ by a redundant global phase.

## Implementation of Arbitrary 2x2 Unitary Matrix via ZYZ Decomposition

Alternatively, the quantum circuit for a given 2x2 unitary matrix can be obtained via the ZYZ decomposition [1]:

$$
\begin{split}
    & \mathcal{U} = e^{i\alpha} \textbf{R}_z(\beta) \textbf{R}_y(\gamma) \textbf{R}_z(\delta) = \\
    & = \left( \begin{matrix} e^{i(\alpha - \beta/2 - \delta/2)} \cos\frac{\gamma}{2} & -e^{i(\alpha - \beta/2 + \delta/2)} \sin\frac{\gamma}{2} \\ e^{i(\alpha + \beta/2 - \delta/2)} \sin\frac{\gamma}{2} & e^{i(\alpha + \beta/2 + \delta/2)} \cos\frac{\gamma}{2} \end{matrix} \right),
\end{split}
$$

Equating this matrix to the general form of a 2x2 unitary matrix stated above gives:

$$\alpha = \frac{\eta}{2} \\
  \beta = \pi + \eta - \textrm{arg}(a) - \textrm{arg}(b)\\
  \gamma = 2 \arctan\Big(\frac{|b|}{|a|}\Big)\\
  \delta = -\pi + \textrm{arg}(b) - \textrm{arg}(a)$$

Let us now define a function that implements the ZYZ decomposition of an arbitrary 2x2 unitary matrix and outputs the corresponding quantum circuit:

In [9]:
def ZYZ_decompose(U):
    '''
    Function that finds parameters of ZYZ decomposition of U
    U: 2x2 unitary matrix [numpy.ndarray]
    '''
    
    # Checking if input is numpy.ndarray
    if not type(U) is np.ndarray:
        sys.exit("Input must be numpy.ndarray")
    
    # Checking if input has right dimensionality
    if not U.shape == (2,2):
        sys.exit("Input must be 2x2 matrix")
        
    # Checking if input is unitary
    if not is_unitary(U):
        sys.exit("Input must be unitary")
    
    a = U[0,0]
    b = U[0,1]
    eta = np.angle(U[1,1]/(np.conjugate(a)))
    
    alpha = 0.5*eta
    beta = cmath.pi + eta - np.angle(a) - np.angle(b)
    gamma = 2*np.arctan(np.absolute(b)/np.absolute(a))
    delta = -cmath.pi + np.angle(b) - np.angle(a)
    
    params = [alpha, beta, gamma, delta]
    
    return params  
    

def ZYZ_circuit(U):
    '''
    Function that finds quantum circuit that implements U based on ZYZ decomposition
    U: 2x2 unitary matrix [numpy.ndarray]
    '''
    
    [alpha, beta, gamma, delta] = ZYZ_decompose(U)
    
    qc = QuantumCircuit(1)
    qc.rz(delta,0)
    qc.ry(gamma,0)
    qc.rz(beta,0)
    
    # Sanity check to accuracy set by f_acc
    f_acc = 1e-6
    
    simulator = Aer.get_backend('unitary_simulator')
    result = execute(qc, simulator).result()
    ZYZ = result.get_unitary(qc)
    
    numerator = (np.trace(np.matmul(np.conjugate(np.transpose(U)), ZYZ)))**2
    denominator = np.trace(np.matmul(np.conjugate(np.transpose(U)), U)) * np.trace(np.matmul(np.conjugate(np.transpose(ZYZ)), ZYZ))
    fidelity_measure = numerator / denominator
        
    if 1 - abs(fidelity_measure) < f_acc:
        return qc
    else:
        sys.exit("Could not obtain ZYZ decomposition to desired accuracy")

We can now check that our function works for an arbitrary unitary matrix:

In [25]:
U = qiskit.quantum_info.random_unitary(2).data
qc = ZYZ_circuit(U)

# Sanity check
simulator = qiskit.Aer.get_backend('unitary_simulator')
result = execute(qc, simulator).result()
output_ZYZ = result.get_unitary(qc)

print(np.divide(output_ZYZ, U))

[[-0.63771222+0.77027471j -0.63771222+0.77027471j]
 [-0.63771222+0.77027471j -0.63771222+0.77027471j]]


## Controlled Single-Qubit Gates via $U_3$ Gate

The controlled version of a unitary 2x2 matrix U can be straightforwardly implemented by taking advantage of the QISKit built-in function _cu3_. There is, however, a caveat that must be considered first: a controlled-U gate is sensitive to the global phase of U, so we cannot disregard global phase factors in the basis gate decomposition of U, as we did earlier when using the $U_3$ gate. We must therefore add an extra parameter, which can be introduced via a $R_z$ gate:

$$
\begin{split}
    & U_3(\theta, \phi, \lambda) R_z(2\mu) = \left( \begin{matrix} \cos(\theta/2) & -e^{i \lambda} \sin(\theta/2) \\ e^{i\phi} \sin(\theta/2) & e^{i(\lambda + \phi)} \cos(\theta/2) \end{matrix} \right)
    \left( \begin{matrix} e^{-i\mu} & 0 \\ 0 & e^{i\mu} \end{matrix} \right) =
    \left( \begin{matrix} e^{-i\mu} \cos(\theta/2) & -e^{i(\lambda + \mu)} \sin(\theta/2) \\ e^{i(\phi - \mu)} \sin(\theta/2) & e^{i(\lambda + \phi + \mu)} \cos(\theta/2) \end{matrix} \right)
\end{split}
$$

Comparing to the general expression for a 2x2 unitary matrix $\mathcal{U}(a,b,\eta)$, the parameters $\theta$, $\phi$, $\lambda$ and $\mu$ are given by:

$$\theta = 2 \arctan\Big(\frac{|b|}{|a|}\Big) \\
  \lambda = -\pi + \textrm{arg}(a) + \textrm{arg}(b)\\
  \phi = \pi + \eta - \textrm{arg}(a) - \textrm{arg}(b)\\
  \mu = -\textrm{arg}(a)$$

The controlled-U gate is obtained by simply taking the controlled versions of both $R_z$ and $U_3$ gates:

<img src="fig2.png" alt="Drawing" style="width: 600px;"/>

In [26]:
def controlled_matrix(U):
    '''
    Function that finds 4x4 matrix that represents controlled-U
    U: 2x2 unitary matrix [numpy.ndarray]
    '''
    
    top_cU = np.hstack((np.eye(2), np.zeros([2,2])))
    bottom_cU = np.hstack((np.zeros([2,2]),U))
    cU = np.vstack((top_cU,bottom_cU))
    
    return cU

def find_params_2(U):
    '''
    Function that finds parameters of U3 and Rz that implement unitary matrix U exactly
    U: Unitary 2x2 matrix [numpy.ndarray]
    '''
    
    # Checking if input is numpy.ndarray
    if not type(U) is np.ndarray:
        sys.exit("Input must be numpy.ndarray")
    
    # Checking if input has right dimensionality
    if not U.shape == (2,2):
        sys.exit("Input must be 2x2 matrix")
        
    # Checking if input is unitary
    if not is_unitary(U):
        sys.exit("Input must be unitary")
    
    a = U[0,0]
    b = U[0,1]
    eta = np.angle(U[1,1]/(np.conjugate(a)))
    
    theta = 2*np.arctan(np.absolute(b)/np.absolute(a))
    lamda = -cmath.pi + np.angle(a) + np.angle(b)
    phi = cmath.pi + eta - np.angle(a) - np.angle(b)
    mu = -np.angle(a)
    
    params = [theta, phi, lamda, mu]
    
    return params

def controlled_gate_circuit(U):
    '''
    Function that outputs U3 gate that implement unitary matrix U
    U: Unitary 2x2 matrix [numpy.ndarray]
    '''
    
    [theta, phi, lamda, mu] = find_params_2(U)
    
    qc = QuantumCircuit(2)
    qc.crz(2*mu,1,0)
    qc.cu3(theta,phi,lamda,1,0)
    
    # Sanity check to accuracy set by f_acc
    f_acc = 1e-6
    
    simulator = Aer.get_backend('unitary_simulator')
    result = execute(qc, simulator).result()
    u3_out = result.get_unitary(qc)
    
    cU = controlled_matrix(U)
    
    numerator = (np.trace(np.matmul(np.conjugate(np.transpose(cU)), u3_out)))**2
    denominator = np.trace(np.matmul(np.conjugate(np.transpose(cU)), cU)) * np.trace(np.matmul(np.conjugate(np.transpose(u3_out)), u3_out))
    fidelity_measure = numerator / denominator
        
    if 1 - abs(fidelity_measure) < f_acc:
        return qc
    else:
        sys.exit("Could not obtain ZYZ decomposition to desired accuracy")

In [34]:
U = qiskit.quantum_info.random_unitary(2).data
qc = controlled_gate_circuit(U)

# Sanity check
simulator = qiskit.Aer.get_backend('unitary_simulator')
result = execute(qc, simulator).result()
output_u3 = result.get_unitary(qc)

cU = controlled_matrix(U)

numerator = (np.trace(np.matmul(np.conjugate(np.transpose(output_u3)), cU)))**2
denominator = np.trace(np.matmul(np.conjugate(np.transpose(output_u3)), output_u3)) * np.trace(np.matmul(np.conjugate(np.transpose(cU)), cU))
fidelity_measure = abs(numerator / denominator)

print(fidelity_measure)

1.0


## Controlled Single-Qubit Gates via ZYZ Decomposition

A single-qubit gate can also be easily converted into its controlled version via the ZYZ decomposition [1]:

<img src="fig1.png">

where $\alpha$, $\beta$, $\gamma$ and $\delta$ are the parameters of the ZYZ decomposition found above.

In [35]:
def controlled_gate_circuit_2(U):
    '''
    Function that finds quantum circuit that implements controlled-U based on ZYZ decomposition
    U: 2x2 unitary matrix [numpy.ndarray]
    '''
    
    [alpha, beta, gamma, delta] = ZYZ_decompose(U)
    
    qc = QuantumCircuit(2)
    qc.rz(0.5*(delta-beta),0)
    qc.cx(1,0)
    qc.rz(-0.5*(delta+beta),0)
    qc.ry(-0.5*gamma,0)
    qc.cx(1,0)
    qc.ry(0.5*gamma,0)
    qc.rz(beta,0)
    qc.rz(alpha,1)
    
    # Sanity check to accuracy set by f_acc
    f_acc = 1e-6
    
    simulator = Aer.get_backend('unitary_simulator')
    result = execute(qc, simulator).result()
    ZYZ = result.get_unitary(qc)
        
    cU = controlled_matrix(U)
    numerator = (np.trace(np.matmul(np.conjugate(np.transpose(cU)), ZYZ)))**2
    denominator = np.trace(np.matmul(np.conjugate(np.transpose(cU)), cU)) * np.trace(np.matmul(np.conjugate(np.transpose(ZYZ)), ZYZ))
    fidelity_measure = numerator / denominator
        
    if 1 - abs(fidelity_measure) < f_acc:
        return qc
    else:
        sys.exit("Could not obtain controlled-U to desired accuracy")

In [40]:
U = qiskit.quantum_info.random_unitary(2).data
qc = controlled_gate_circuit_2(U)

# Sanity check
simulator = qiskit.Aer.get_backend('unitary_simulator')
result = execute(qc, simulator).result()
output_ZYZ = result.get_unitary(qc)

qc_init = QuantumCircuit(2)
cU = controlled_matrix(U)
opts = {"initial_unitary": cU}
result_init = execute(qc_init, simulator, backend_options=opts).result()
output_init = result_init.get_unitary(qc_init)

numerator = (np.trace(np.matmul(np.conjugate(np.transpose(output_ZYZ)), output_init)))**2
denominator = np.trace(np.matmul(np.conjugate(np.transpose(output_ZYZ)), output_ZYZ)) * np.trace(np.matmul(np.conjugate(np.transpose(output_init)), output_init))
fidelity_measure = abs(numerator / denominator)

print(fidelity_measure)

1.0


## References

[1] M. A. Nielsen and I. L. Chuang, _Quantum Computation and Quantum Information - 10th Anniversary Edition_ (Cambridge University Press, 2010)